# CE888 - Lab 4 

- Create a new ipython notebook
- Load the data from the file jester-data-1.csv
- The data is from http://eigentaste.berkeley.edu/dataset/
   and it contains the ratings of 101 jokes from 24,983 users
- The jokes are here
- Label approx 10% of the dataset cells as 99, to denote they are part of the validation set. Keep the the actual values of the cells so you can use them later.
- Use latent factor modelling to infer the hidden ratings of the users (they are labeled as ”99” in the dataset) on the training set
- Calculate the performance of the algorithm on the validation dataset
- Change hyper-parameters (i.e. learning rates, number of iterations, number of latent factors etc) as needed so you can get good results
- Report the MSE on the test dataset
- (Bonus) Use pandas to find the best and the worst rated joke

In [1]:
# Read the data file (jester-data-1.csv)

import pandas as pd
import numpy as np
import random as rand
np.set_printoption(precision = 3)

df = pd.read_csv('jester-data-1.csv')

df_copy = df.copy()

In [2]:
df.describe()

,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,...,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
count,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,...,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000
mean,72.467417,34.403125,27.094822,33.265435,36.246713,0.427554,19.343233,-0.401425,-0.616301,36.230231,...,61.464367,60.278340,60.420157,59.058023,58.748286,58.539484,58.019793,57.059971,55.751626,60.136534
std,21.860035,46.707230,44.228739,46.747682,48.813136,5.472535,37.861118,5.741420,5.008903,48.228481,...,47.334898,47.946774,47.360166,48.196506,48.311024,48.130845,48.163962,48.717708,49.237364,47.913859
min,36.000000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,53.000000,-0.150000,-2.180000,-1.120000,-3.350000,-3.690000,-0.390000,-5.290000,-4.610000,-1.750000,...,4.760000,3.590000,4.610000,3.250000,3.160000,3.300000,3.590000,2.670000,1.322500,3.930000
50%,72.000000,5.050000,3.690000,4.660000,3.450000,1.120000,3.540000,-0.050000,-0.290000,4.030000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
75%,100.000000,99.000000,99.000000,99.000000,99.000000,4.470000,8.010000,3.930000,3.060000,99.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
max,100.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [8]:
# Label approx 10% of the dataset cells as 99

users = df_copy.shape[0]
jokes = df_copy.shape[1]
counts = users * jokes

counts_unspecified = (df_copy.iloc[:,:] == 99).sum().sum()
counts_unspecified_percent = (counts_unspecified / counts) * 100

counts_specified = counts - counts_unspecified
counts_to_change = round(counts_specified * 0.1).astype(int)

row = 0
col = 0
for x in range(0, counts_to_change):  
    while (df_copy.iloc[row, col]) == 99:
        row = rand.randint(0, users - 1)
        col = rand.randint(0, jokes - 1)
    df_copy.at[row, col] = 99    

df_copy.head()

,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,...,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25,0
0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07,99.0
1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,NaN
2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00,NaN
3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60,NaN
4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,...,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45,NaN


In [9]:
f = 3

# Use latent factor modeling to infer the hidden ratings of the users
# (they are labeled as "99" in the dataset) on the training set
latent_item_feature = np.random.random((jokes, f))
latent_user_preference = np.random.random((users, f))


In [16]:
# Calculate the performance of the algorithm on the validation dataset

def predict_rating(user_row, joke_col):
    user_values = latent_user_preferences[user_row]
    item_values = latent_item_features[joke_col]
    return user_values.dot(item_values)

# Training function.
def train(user_row, item_row, rating, alpha = 0.001):

    err = alpha * (rating - predict_rating(user_row, item_row))
    latent_user_preferences[user_row] += err * latent_item_features[item_row]
    latent_item_features[item_row] += err * latent_user_preferences[user_row]
    return err

def sgd(iterations = 100):

    for iteration in range(0,iterations):
        training_error = []
        for user_row in range(0, 100):
            for joke_col in range(0, jokes):
                rating = df_copy.iloc[user_row][joke_col]
                if(not np.isnan(rating) and rating < 99):
                    training_error.append(train(user_row, joke_col, rating))
    mse = (np.array(training_error) ** 2).mean()          
    print (mse)

In [11]:
print('Latent item features = ' + repr(latent_item_features))
print('Latent user preferences = ' + repr(latent_user_preferences))

Latent item features = array([[0.691, 0.495, 0.895],
       [0.852, 0.793, 0.577],
       [0.621, 0.142, 0.005],
       [0.901, 0.203, 0.394],
       [0.101, 0.805, 0.282],
       [0.178, 0.846, 0.531],
       [0.946, 0.023, 0.374],
       [0.74 , 0.753, 0.752],
       [0.021, 0.969, 0.019],
       [0.123, 0.648, 0.217],
       [0.1  , 0.999, 0.574],
       [0.355, 0.507, 0.986],
       [0.778, 0.866, 0.128],
       [0.439, 0.157, 0.395],
       [0.831, 0.47 , 0.819],
       [0.377, 0.039, 0.158],
       [0.15 , 0.361, 0.129],
       [0.052, 0.349, 0.56 ],
       [0.466, 0.656, 0.603],
       [0.375, 0.794, 0.393],
       [0.694, 0.981, 0.623],
       [0.144, 0.329, 0.512],
       [0.21 , 0.522, 0.979],
       [0.395, 0.623, 0.14 ],
       [0.304, 0.407, 0.706],
       [0.458, 0.183, 0.316],
       [0.842, 0.421, 0.395],
       [0.545, 0.227, 0.86 ],
       [0.502, 0.186, 0.05 ],
       [0.088, 0.926, 0.938],
       [0.133, 0.822, 0.365],
       [0.431, 0.454, 0.804],
       [0.351, 0.

In [15]:
#MSE

validation_error = []
for user in range(0, 100):
    for joke in range(0, jokes):
        rating_training_data = df.iloc[user][joke]
        rating_validation_data = df_copy.iloc[user][joke]
        if (not rating_training_data == rating_validation_data):
            prediction = predict_rating(user, joke)
            validation_error.append(rating_training_data - predict_rating(user, joke))
            #print("Expected: " + str(rating_training_data) + "\tActual: " + str(predict_rating(user, joke)))
            
print ("Validation MSE: " + str((np.array(validation_error) ** 2).mean()))

KeyError: 1